# LTN for Knowledge-aware Intrusion Detection
This implementation is based on the example implementation from [logictensornetworks](https://github.com/logictensornetworks/logictensornetworks)

In [2]:
import os
os.environ["NUMEXPR_MAX_THREADS"] = "16"

import logging; logging.basicConfig(level=logging.INFO)
import tensorflow as tf
import pandas as pd
import ltn

2024-04-27 23:34:59.376156: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
# Define meta variables
test_fraction = 0.7
training_fraction = 1 - test_fraction
batch_size = 10000

# Specify domain knowledge the IDS2017 dataset description.

In [4]:
# list of all ips in the victim network in IDS 2017
own_win_servers = [
    "192.168.10.3",
]
own_linux_servers = [
    "192.168.10.50",
    "205.174.165.68",
    "192.168.10.51",
    "205.174.165.66",
]

own_ubuntu_pcs = [
    "192.168.10.19",
    "192.168.10.17",
    "192.168.10.16",
    "192.168.10.12",
]
own_windows_pcs = [
    "192.168.10.9",
    "192.168.10.5",
    "192.168.10.8",
    "192.168.10.14",
    "192.168.10.15",
]
own_mac_pcs = [
    "192.168.10.25",
]

own_server = own_win_servers + own_linux_servers
own_pcs = own_ubuntu_pcs + own_windows_pcs + own_mac_pcs
own_ips = own_server + own_pcs
webserver_ips = ["192.168.10.50", "192.168.10.51"]

# Read the and partition dataset

In [5]:
with open("IDS2017_subset.csv", "r") as file:
    dataset = pd.read_csv(file, delimiter=",")

print(dataset.head(3))

       Source IP   Source Port  Destination IP   Destination Port   Protocol  \
0  192.168.10.50         33898    192.168.10.3                389          6   
1  192.168.10.50         33904    192.168.10.3                389          6   
2        8.6.0.1             0         8.0.6.4                  0          0   

    Flow Duration  Total Length of Fwd Packets   Total Length of Bwd Packets  \
0       113095465                         9668                       10012.0   
1       113473706                        11364                       12718.0   
2       119945515                            0                           0.0   

    Fwd Header Length   Bwd Header Length  Fwd PSH Flags  FIN Flag Count  \
0                1536                 768              1               0   
1                2176                1280              1               0   
2                   0                   0              0               0   

    Bwd Packet Length Min  Init_Win_bytes_forward   I

In [6]:
# Normalize all columns between 0 and 1
for column in dataset.columns:
    if column not in [" Label", " Source IP", " Destination IP"]:
        dataset[column] = (dataset[column] - dataset[column].min()) / (
            dataset[column].max() - dataset[column].min()
        )
# Port is a categorical variable, but encoding it as a number works partly 

In [7]:
dataset["is_webserver"] = dataset[" Destination IP"].apply(
    lambda x: 1 if x in webserver_ips else 0
)
dataset["from_own_network"] = dataset[" Source IP"].apply(
    lambda x: 1 if x in own_ips else 0
)
dataset["to_own_network"] = dataset[" Destination IP"].apply(
    lambda x: 1 if x in own_ips else 0
)
## remove the columns " Source IP" and " Destination IP"
dataset = dataset.drop(columns=[" Source IP", " Destination IP"])

In [8]:
# partition the dataset into 4 classes
benign = dataset[dataset[" Label"] == "BENIGN"]
brute_force = dataset[dataset[" Label"] == "Web Attack � Brute Force"]
xss = dataset[dataset[" Label"] == "Web Attack � XSS"]

# 70% of each class for training and the remainding 30% for testing
benign_train = benign.sample(frac=test_fraction)
benign_test = benign.drop(benign_train.index)
brute_force_train = brute_force.sample(frac=test_fraction)
brute_force_test = brute_force.drop(brute_force_train.index)
xss_train = xss.sample(frac=test_fraction)
xss_test = xss.drop(xss_train.index)

# randomly conatenate all the dataframes
training_set = pd.concat([benign_train, brute_force_train, xss_train])
test_set = pd.concat([benign_test, brute_force_test, xss_test])
# shuffle the dataframes
training_set = training_set.sample(frac=1).reset_index(drop=True)
test_set = test_set.sample(frac=1).reset_index(drop=True)


labels_train = training_set.pop(" Label")
labels_test = test_set.pop(" Label")

## create a metadata tensor this info is not shown to the neural network
metadata_train = training_set[["is_webserver", "from_own_network", "to_own_network"]]
metadata_test = test_set[["is_webserver", "from_own_network", "to_own_network"]]
training_set = training_set.drop(columns=["is_webserver", "from_own_network", "to_own_network"])
test_set = test_set.drop(columns=["is_webserver", "from_own_network", "to_own_network"])

# convert the labels to a numeric value
labels_train = labels_train.replace("BENIGN", 0)
labels_train = labels_train.replace("Web Attack � Brute Force", 1)
labels_train = labels_train.replace("Web Attack � XSS", 2)
labels_test = labels_test.replace("BENIGN", 0)
labels_test = labels_test.replace("Web Attack � Brute Force", 1)
labels_test = labels_test.replace("Web Attack � XSS", 2)

# Create batches
ds_train = tf.data.Dataset.from_tensor_slices(
    (training_set, labels_train, metadata_train)
).batch(batch_size)
ds_test = tf.data.Dataset.from_tensor_slices(
    (test_set, labels_test, metadata_test)
).batch(batch_size)

    

2024-04-27 23:35:01.356867: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


# Define LTN components
Define predicate `P(x,class)`

A fully connected MLP (16,16,8)

In [9]:
class MLP(tf.keras.Model):
    """Model that returns logits."""

    def __init__(self, n_classes, hidden_layer_sizes=(16, 16, 8)):
        super(MLP, self).__init__()
        self.denses = [
            tf.keras.layers.Dense(s, activation="elu") for s in hidden_layer_sizes
        ]
        self.dense_class = tf.keras.layers.Dense(n_classes)
        self.dropout = tf.keras.layers.Dropout(0.2)

    def call(self, inputs, training=False):
        x = inputs[0]
        for dense in self.denses:
            x = dense(x)
            x = self.dropout(x, training=training)
        return self.dense_class(x)


logits_model = MLP(3)
p = ltn.Predicate.FromLogits(
    logits_model, activation_function="softmax", with_class_indexing=True
)

Define Claseses `Benign, Brute force, Xss`

In [10]:
class_benign = ltn.Constant(0, trainable=False)
class_brute_force = ltn.Constant(1, trainable=False)
class_xss = ltn.Constant(2, trainable=False)

def get_class_name(class_index):
    if class_index == 0:
        return "BENIGN"
    elif class_index == 1:
        return "Brute Force"
    else:
        return "XSS"

# Define operators

In [11]:
Not = ltn.Wrapper_Connective(ltn.fuzzy_ops.Not_Std())
And = ltn.Wrapper_Connective(ltn.fuzzy_ops.And_Prod())
Or = ltn.Wrapper_Connective(ltn.fuzzy_ops.Or_ProbSum())
Implies = ltn.Wrapper_Connective(ltn.fuzzy_ops.Implies_Reichenbach())
Forall = ltn.Wrapper_Quantifier(ltn.fuzzy_ops.Aggreg_pMeanError(p=2),semantics="forall")

# Define Real logic statements 

<p align="center"> <i> Ground with labeled data </i> </p>

$$ \forall x\_l\_benign P(x\_l\_benign, Class\_Benign) $$
$$ \forall x\_l\_bruteForce P(x\_l\_bruteForce, Class\_BruteForce) $$
$$ \forall x\_l\_xss P(x\_l\_xss, Class\_XSS) $$

---

<p align="center"> <i>Any non-websever cannot be classified as having a web-attack</i> </p>

$$ \forall x\_not\_webserver \neg (P(x\_not\_webserver, Class\_bruteForce) \wedge P(x\_not\_webserver, Class\_XSS)) $$

In [12]:
formula_aggregator = ltn.Wrapper_Formula_Aggregator(
    ltn.fuzzy_ops.Aggreg_pMeanError(p=2)
)


@tf.function
def axioms(features, labels, metadata, training=False):     
    x_Benign = ltn.Variable("x_Benign", features[labels == 0])
    x_Brute_Force = ltn.Variable("x_Brute_Force", features[labels == 1])
    x_XSS = ltn.Variable("x_XSS", features[labels == 2])


    # traffic from webserver
    ws_index = list(metadata_train.columns).index("is_webserver")
    x_not_webserver = ltn.Variable(
        "x_not_webserver", features[metadata[:, ws_index] == 0]
    )

    axioms = [
        Forall(x_Benign, p([x_Benign, class_benign], training=training)),
        Forall(x_Brute_Force, p([x_Brute_Force, class_brute_force], training=training)),
        Forall(x_XSS, p([x_XSS, class_xss], training=training)),
        # only webserver traffic is brute force or xss
        Forall(
            x_not_webserver,
            Not(
                Or(
                    p([x_not_webserver, class_brute_force], training=training),
                    p([x_not_webserver, class_xss], training=training),
                )
            ),
        )
    ]
    sat_level = formula_aggregator(axioms).tensor
    return sat_level

Initialize all layers and the static graph

In [13]:
for features, labels, metadata in ds_test:
    print("Initial sat level %.5f"%axioms(features, labels, metadata))
    break


2024-04-27 23:35:01.510457: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_2' with dtype int64 and shape [51104,3]
	 [[{{node Placeholder/_2}}]]


Initial sat level 0.31719


# Training

In [14]:
metrics_dict = {
    'train_sat_kb': tf.keras.metrics.Mean(name='train_sat_kb'),
    'test_sat_kb': tf.keras.metrics.Mean(name='test_sat_kb'),
    'train_accuracy': tf.keras.metrics.CategoricalAccuracy(name="train_accuracy"),
    'test_accuracy': tf.keras.metrics.CategoricalAccuracy(name="test_accuracy")
}

In [15]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
@tf.function
def train_step(features, labels, metadata):
    # sat and update
    with tf.GradientTape() as tape:
        sat = axioms(features, labels, metadata, training=True)
        loss = 1.-sat
    gradients = tape.gradient(loss, p.trainable_variables)
    optimizer.apply_gradients(zip(gradients, p.trainable_variables))

    sat = axioms(features, labels, metadata) # compute sat without dropout
    metrics_dict['train_sat_kb'](sat)
    # accuracy
    predictions = logits_model([features])
    metrics_dict['train_accuracy'](tf.one_hot(labels,3),predictions)
    
@tf.function
def test_step(features, labels, metadata):
    # sat
    sat = axioms(features, labels, metadata)
    metrics_dict['test_sat_kb'](sat)
    # accuracy
    predictions = logits_model([features])
    metrics_dict['test_accuracy'](tf.one_hot(labels,3),predictions)

Train

In [16]:
import importlib
import commons
importlib.reload(commons)
EPOCHS = 200
 
commons.train(
    EPOCHS,
    metrics_dict,
    ds_train,
    ds_test,
    train_step,
    test_step,
    csv_path="training_results.csv",
    track_metrics=20
)

2024-04-27 23:35:05.707801: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_2' with dtype int64 and shape [119240,3]
	 [[{{node Placeholder/_2}}]]
2024-04-27 23:35:06.321130: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'features' with dtype double and shape [10000,15]
	 [[{{node features}}]]
2024-04-27 23:35:07.016683: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'features' with dtype double and shape [10000,15]
	 [[{

Epoch 0, train_sat_kb: 0.3372, test_sat_kb: 0.3516, train_accuracy: 0.1014, test_accuracy: 0.1931
Epoch 20, train_sat_kb: 0.5736, test_sat_kb: 0.5549, train_accuracy: 0.9185, test_accuracy: 0.9223
Epoch 40, train_sat_kb: 0.5824, test_sat_kb: 0.5607, train_accuracy: 0.9296, test_accuracy: 0.9276
Epoch 60, train_sat_kb: 0.5892, test_sat_kb: 0.5667, train_accuracy: 0.9489, test_accuracy: 0.9471
Epoch 80, train_sat_kb: 0.5916, test_sat_kb: 0.5684, train_accuracy: 0.9510, test_accuracy: 0.9505
Epoch 100, train_sat_kb: 0.5934, test_sat_kb: 0.5692, train_accuracy: 0.9524, test_accuracy: 0.9509
Epoch 120, train_sat_kb: 0.5953, test_sat_kb: 0.5708, train_accuracy: 0.9525, test_accuracy: 0.9518
Epoch 140, train_sat_kb: 0.5973, test_sat_kb: 0.5728, train_accuracy: 0.9538, test_accuracy: 0.9506
Epoch 160, train_sat_kb: 0.5997, test_sat_kb: 0.5742, train_accuracy: 0.9541, test_accuracy: 0.9542
Epoch 180, train_sat_kb: 0.6026, test_sat_kb: 0.5769, train_accuracy: 0.9561, test_accuracy: 0.9547


# Calculate the performance

In [17]:
import math

predictions = logits_model([tf.convert_to_tensor(test_set)])
confusion_matrix = tf.math.confusion_matrix(
    labels_test, tf.argmax(predictions, axis=1), num_classes=3
)


print("|    Label    |  TP    |   FP  |   FN  | Precision | Recall |   F1  |  MCC  |")
print("|-------------|--------|-------|-------|-----------|--------|-------|-------|")

# for each class, compute the precision, recall and f1 score
for i in range(3):
    tp = int((confusion_matrix[i, i]))
    fn = int(tf.reduce_sum(confusion_matrix[i, :]) - confusion_matrix[i, i])
    fp = int(tf.reduce_sum(confusion_matrix[:, i]) - confusion_matrix[i, i])
    tn = int(tf.reduce_sum(confusion_matrix) - tp - fp - fn)


    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    f1 = (2 * precision * recall) / (precision + recall)
    mcc = (tp * tn - fp * fn) / math.sqrt(((tp + fp) * (tp + fn) * (tn + fp) * (tn + fn)))
    print(
        f"| {get_class_name(i):11} | {tp:6} | {fp:5} | {fn:5} | {precision:9.3f} | {recall:6.3f} | {f1:5.3f} | {mcc:3.3f} |"
    )

|    Label    |  TP    |   FP  |   FN  | Precision | Recall |   F1  |  MCC  |
|-------------|--------|-------|-------|-----------|--------|-------|-------|
| BENIGN      |  48341 |    85 |  2115 |     0.998 |  0.958 | 0.978 | 0.415 |
| Brute Force |    287 |  1830 |   165 |     0.136 |  0.635 | 0.223 | 0.281 |
| XSS         |    126 |   435 |    70 |     0.225 |  0.643 | 0.333 | 0.376 |
